In [1]:
import xarray as xr
import numpy as np
import torch
import pytorch_lightning as pl
import torch.nn as nn
#import xrft
import matplotlib.pyplot as plt
#import pandas as pd
from tqdm import tqdm
import os
import glob
from pathlib import Path
from typing import Union, List
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import torch.nn.functional as F
import cartopy.crs as ccrs
import pickle

In [2]:
from src.utils import load_ssf_acoustic_variables
from src.data import TrainingItem, BaseDatamodule
from src.acoustic_predictor import  AcousticPredictor
from src.autoencoder import AutoEncoder

In [3]:
pl.seed_everything(333)

Seed set to 333


333

In [4]:
sound_speed_path = "/DATASET/eNATL/eNATL60_BLB002_sound_speed_regrid_0_1000m.nc"
ecs_path = "/DATASET/eNATL/eNATL60_BLB002_cutoff_freq_regrid_0_1000m.nc"

In [5]:
ss_da = xr.open_dataarray(sound_speed_path)
ecs_da = xr.open_dataset(ecs_path)['ecs']

In [6]:
x_min = 1459.0439165829073
x_max = 1545.8698054910844

In [7]:
gpu = None
    
if torch.cuda.is_available() and gpu is not None:
##This may not be necessary outside the notebook
    dev = f"cuda:{gpu}"
else:
    dev = "cpu"

device = torch.device(dev)



In [113]:
ae_path = "/homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs/AE_without_AP/4_4_sigmoid_lr_0.001/"
model_ae_path = glob.glob(f"{ae_path}/**/*.ckpt",recursive=True)[-1]
pickle_ae_path = glob.glob(f"{ae_path}/**/*.pickle",recursive=True)[-1]
print(model_ae_path)
#arch_shape = '_'.join(ae_path.split('/')[-1].split('_')[:2])
arch_shape = '4_4'


model_ap_path= "/homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs/accoustic_predictor/2024-04-04_15-22"
model_ap_path = glob.glob(f"{model_ap_path}/**/*.ckpt",recursive=True)[0]
acoustic_predictor = AcousticPredictor.load_from_checkpoint(checkpoint_path = model_ap_path, input_depth= 107)
acoustic_predictor.eval()

print(model_ap_path)

/homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs/AE_without_AP/4_4_sigmoid_lr_0.001/2024-04-08_19-05/checkpoints/val_loss=0.16-epoch=07.ckpt
/homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs/accoustic_predictor/2024-04-04_15-22/checkpoints/val_loss=0.02-epoch=970.ckpt


In [23]:
acoustic_predictor.state_dict()['conv1.block.0.weight'][:8,10,0,1]

tensor([-0.1397,  0.2383, -0.0654, -0.0264, -0.1512,  0.0320, -0.6185, -0.9975],
       device='cuda:0')

In [114]:

model_ae = AutoEncoder.load_from_checkpoint(model_ae_path,
                                            x_min = x_min, 
                                            x_max = x_max,
                                            arch_shape = arch_shape,
                                            acoustic_predictor= acoustic_predictor, 
                                            accoustic_training = False
                                            ).to(device)


    
# for param in model_ae.acoustic_predictor.parameters():
#     param.requires_grad = False

model_ae.eval()

AutoEncoder(
  (acoustic_predictor): AcousticPredictor(
    (conv1): ConvBlock(
      (block): Sequential(
        (0): Conv2d(107, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (conv2): ConvBlock(
      (block): Sequential(
        (0): Conv2d(96, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (conv3): ConvBlock(
      (block): Sequential(
        (0): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (conv4): ConvBlock(
      (block): Sequential(
        (0): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, mo

In [24]:
acoustic_predictor.state_dict()['conv1.block.0.weight'][:8,10,0,1]

tensor([-0.1397,  0.2383, -0.0654, -0.0264, -0.1512,  0.0320, -0.6185, -0.9975],
       device='cuda:0')

In [27]:
model_ae.acoustic_predictor.state_dict()['conv1.block.0.weight'][:8,10,0,1]

tensor([-0.1212,  0.2829, -0.0650, -0.0221, -0.2054,  0.0166, -0.5908, -0.9871],
       device='cuda:0')

In [34]:


model_ap_path = '/homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs/accoustic_predictor/2024-04-08_16-27/checkpoints/val_loss=0.04-epoch=03.ckpt'
model_ap_path = '/homes/o23gauvr/Documents/thèse/code/FASCINATION/outputs/accoustic_predictor/2024-04-04_15-22/checkpoints/val_loss=0.02-epoch=970.ckpt'
acoustic_predictor = AcousticPredictor.load_from_checkpoint(checkpoint_path = model_ap_path,
                                                            input_depth= 107)
print(acoustic_predictor.state_dict()['conv1.block.0.weight'][:8,10,0,1])

acoustic_predictor.eval()

print(acoustic_predictor.state_dict()['conv1.block.0.weight'][:8,10,0,1])


for param in acoustic_predictor.parameters():
    param.requires_grad = False

model_ae = AutoEncoder.load_from_checkpoint(model_ae_path,
                                            x_min = x_min, 
                                            x_max = x_max,
                                            arch_shape = arch_shape,
                                            acoustic_predictor= acoustic_predictor, 
                                            accoustic_training = False,
                                            ).to(device)



for param in model_ae.acoustic_predictor.parameters():
    param.requires_grad = False

model_ae.eval()


print(acoustic_predictor.state_dict()['conv1.block.0.weight'][:8,10,0,1])
print(model_ae.acoustic_predictor.state_dict()['conv1.block.0.weight'][:8,10,0,1])

tensor([-0.1397,  0.2383, -0.0654, -0.0264, -0.1512,  0.0320, -0.6185, -0.9975],
       device='cuda:0')
tensor([-0.1397,  0.2383, -0.0654, -0.0264, -0.1512,  0.0320, -0.6185, -0.9975],
       device='cuda:0')
tensor([-0.1212,  0.2829, -0.0650, -0.0221, -0.2054,  0.0166, -0.5908, -0.9871])
tensor([-0.1212,  0.2829, -0.0650, -0.0221, -0.2054,  0.0166, -0.5908, -0.9871])


In [35]:
torch.load(model_ae_path)['state_dict']['acoustic_predictor.conv1.block.0.weight'][:8,10,0,1]

tensor([-0.1212,  0.2829, -0.0650, -0.0221, -0.2054,  0.0166, -0.5908, -0.9871],
       device='cuda:0')

In [36]:
# y_min_axs = -20
# y_max_axs = 1020
# x_min_axs = 1450
# x_max_axs = 1550

# Generate Dataloaders

In [37]:
dl_kw = {"batch_size": 4, "num_workers": 4}
dl_kw

{'batch_size': 4, 'num_workers': 4}

In [38]:
def test_dataloader_normalization(dataloader):
    
    for batch_number, batch in enumerate(dataloader):
        
        errors = []
        if batch.input.min() < -1:
            errors.append("Minimum value of input is less than -1.")
        if batch.input.max() > 1:
            errors.append("Maximum value of input is greater than 1.")
        if batch.tgt[:,0,:,:].min() < -1:
            errors.append("Minimum value of target ECS is less than -1.")
        if batch.tgt[:,0,:,:].max() > 1:
            errors.append("Maximum value of target ECS is greater than 1.")
        if abs(batch.input.max() - batch.input.min()) > 2:
            errors.append("Absolute difference between input max and min is greater than 2.")
        if abs(batch.tgt[:,0,:,:].max() - batch.tgt[:,0,:,:].min()) > 2:
            errors.append("Absolute difference between target ECS max and min is greater than 2.")
        
        if errors:
            error_message = "Validation failed for batch {}: {}".format(batch_number, "\n ".join(errors))
            raise ValueError(error_message)

In [39]:
def generate_dataloader(sound_speed_path: str,
                        ecs_path: str,
                        dl_kw: dict,
                        x_min: Union[float, None] = None,
                        x_max: Union[float, None] = None,
                        verbose:float = True
                        ):
    
    
    

    dm_dict = dict()
    phases = ['train','val','test']    

    input_da = load_ssf_acoustic_variables(sound_speed_path,ecs_path)

    dm = BaseDatamodule(input_da, dl_kw, x_min=x_min, x_max =x_max)

    for phase in tqdm(phases, unit = "phase", desc = "Genereting Dataloaders on all splits"):
        
        if verbose:
            print("phase:", phase)


        if phase == 'train':
            if verbose:
                print("\t Traint/Val setup")
            dm.setup(stage = 'fit')
            
            if verbose:
                print("\tGenerating dataloader")
            dataloader = dm.train_dataloader()
        
        
        elif phase == "val":
            if verbose:
                print("\tGenerating dataloader")
            dataloader = dm.val_dataloader()
        
        
        elif phase == 'test':  
            if verbose:
                print("\t test setup")
            dm.setup(stage = 'test')
            
            if verbose:
                print("\tGenerating dataloader")
            dataloader = dm.test_dataloader()


        if verbose:
            print("Testing dataloader normalization")
        test_dataloader_normalization(dataloader)


        dm_dict[phase] = dataloader
        
 
    return dm_dict
 


In [40]:
dm_dict = generate_dataloader(sound_speed_path,
                              ecs_path,
                              dl_kw,
                              x_min,
                              x_max)



Genereting Dataloaders on all splits:   0%|          | 0/3 [00:00<?, ?phase/s]

phase: train
	 Traint/Val setup
	Generating dataloader
Testing dataloader normalization


Genereting Dataloaders on all splits:  33%|███▎      | 1/3 [05:49<11:39, 349.60s/phase]

phase: val
	Generating dataloader
Testing dataloader normalization


Genereting Dataloaders on all splits:  67%|██████▋   | 2/3 [05:51<02:25, 145.31s/phase]

phase: test
	 test setup
	Generating dataloader
Testing dataloader normalization


Genereting Dataloaders on all splits: 100%|██████████| 3/3 [05:55<00:00, 118.63s/phase]


We have checked that for the val and test, first batch matches the one obtained through main.py. Test is only diffreent because of the shuffle operation

# Get metrics

## AE RMSE

In [115]:
val_ae_loss = []
with torch.no_grad():
    for batch in dm_dict['val']:
        x,y = batch
        output = model_ae(x)
        loss = torch.sqrt(nn.MSELoss()(output,x))
        val_ae_loss.append(loss)

In [116]:
test_ae_loss = []
with torch.no_grad():
    for batch in dm_dict['test']:
        x,y = batch
        output = model_ae(x)
        loss = torch.sqrt(nn.MSELoss()(output*(x_max - x_min)+x_min, x*(x_max - x_min)+x_min))
        test_ae_loss.append(loss)

stable models that matches model.test_step() through trainer:

    without AE:
        pca_50
        pca_107
        pca_4
        32_120
        4_15
        4_4

In [117]:
val_ae_loss

[tensor(0.1571),
 tensor(0.1531),
 tensor(0.1559),
 tensor(0.1571),
 tensor(0.1586),
 tensor(0.1575),
 tensor(0.1568),
 tensor(0.1562),
 tensor(0.1564),
 tensor(0.1576),
 tensor(0.1559),
 tensor(0.1540),
 tensor(0.1557),
 tensor(0.1554),
 tensor(0.1550),
 tensor(0.1552),
 tensor(0.1559),
 tensor(0.1574),
 tensor(0.1605)]

In [118]:
model_ae.acoustic_predictor.state_dict()['conv1.block.0.weight'][:8,10,0,1]

tensor([-0.1397,  0.2383, -0.0654, -0.0264, -0.1512,  0.0320, -0.6185, -0.9975])

In [119]:
model_ae.state_dict()['acoustic_predictor.conv1.block.0.weight'][:8,10,0,1]

tensor([-0.1397,  0.2383, -0.0654, -0.0264, -0.1512,  0.0320, -0.6185, -0.9975])

In [120]:
model_ae.state_dict()

OrderedDict([('acoustic_predictor.conv1.block.0.weight',
              tensor([[[[ 5.8341e-01, -1.0269e-01, -2.5493e-01],
                        [ 7.6659e-01, -1.2543e-01, -3.5631e-01],
                        [ 4.7546e-01, -3.0943e-01, -5.6432e-01]],
              
                       [[ 4.9355e-01, -2.6382e-01, -4.1156e-01],
                        [ 6.4325e-01, -2.6164e-01, -4.4187e-01],
                        [ 3.1195e-01, -5.0231e-01, -7.6219e-01]],
              
                       [[ 4.0487e-01, -3.5066e-01, -5.1736e-01],
                        [ 5.1247e-01, -3.3524e-01, -5.3816e-01],
                        [ 1.4334e-01, -6.6436e-01, -9.0201e-01]],
              
                       ...,
              
                       [[ 6.4741e-02, -2.1774e-01,  9.9246e-02],
                        [-1.2275e+00, -1.4063e+00, -4.3278e-01],
                        [-2.4975e+00, -2.4743e+00, -1.8139e+00]],
              
                       [[-7.8928e-03, -3.0081e-01,  4.32

model_ae.state_dict() is identical to self.state_dict() in AutoEncoder

In [121]:
x[0,0,25:28,25:28]

tensor([[0.7151, 0.7148, 0.7128],
        [0.7159, 0.7147, 0.7125],
        [0.7166, 0.7150, 0.7129]])

In [122]:
model_ae.acoustic_predictor(x)[0,0,25:28,25:28]

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [123]:
model_ae(x)[0,0,25:28,25:28]

tensor([[0.7940, 0.7942, 0.7939],
        [0.7941, 0.7947, 0.7926],
        [0.7939, 0.7917, 0.7933]], grad_fn=<SliceBackward0>)

In [124]:
encoded = model_ae.encoder(x)
encoded[0,:3,25:28,25:28]

tensor([], size=(3, 0, 0), grad_fn=<SliceBackward0>)

In [125]:
model_ae.forward(x)[0,0,25:28,25:28]

tensor([[0.7940, 0.7942, 0.7939],
        [0.7941, 0.7947, 0.7926],
        [0.7939, 0.7917, 0.7933]], grad_fn=<SliceBackward0>)

In [126]:
model_ae.decoder(encoded)[0,0,25:28,25:28]

tensor([[0.7940, 0.7942, 0.7939],
        [0.7941, 0.7947, 0.7926],
        [0.7939, 0.7917, 0.7933]], grad_fn=<SliceBackward0>)

In [127]:
test_ae_loss

[tensor(13.7344),
 tensor(13.8605),
 tensor(13.7346),
 tensor(13.5681),
 tensor(13.6238),
 tensor(13.6684),
 tensor(13.5385),
 tensor(13.1939),
 tensor(13.6254)]

In [128]:
np.mean(test_ae_loss)

13.616409

In [129]:
x.isnan().any()

tensor(False)

In [130]:
torch.sqrt(nn.MSELoss()(output*(x_max - x_min)+x_min, x*(x_max - x_min)+x_min))

tensor(13.6254)

In [131]:
output.shape

torch.Size([4, 107, 240, 240])

In [132]:
x,y = next(iter(dm_dict['test']))

In [133]:
output.min()

tensor(0.2323)

In [134]:
x[0,:3,25:28,25:28]

tensor([[[0.8519, 0.8523, 0.8522],
         [0.8510, 0.8508, 0.8506],
         [0.8504, 0.8502, 0.8501]],

        [[0.8521, 0.8525, 0.8524],
         [0.8512, 0.8510, 0.8508],
         [0.8506, 0.8504, 0.8503]],

        [[0.8524, 0.8528, 0.8527],
         [0.8514, 0.8513, 0.8511],
         [0.8509, 0.8507, 0.8505]]])

In [135]:
output = model_ae(x)

output[0,:3,25:28,25:28]

tensor([[[0.7940, 0.7942, 0.7939],
         [0.7941, 0.7947, 0.7926],
         [0.7939, 0.7917, 0.7933]],

        [[0.7949, 0.7941, 0.7949],
         [0.7938, 0.7931, 0.7939],
         [0.7932, 0.7930, 0.7923]],

        [[0.7940, 0.7938, 0.7950],
         [0.7948, 0.7952, 0.7963],
         [0.7946, 0.7938, 0.7944]]], grad_fn=<SliceBackward0>)

In [136]:
model_ae.forward(x)[0,:3,25:28,25:28]

tensor([[[0.7940, 0.7942, 0.7939],
         [0.7941, 0.7947, 0.7926],
         [0.7939, 0.7917, 0.7933]],

        [[0.7949, 0.7941, 0.7949],
         [0.7938, 0.7931, 0.7939],
         [0.7932, 0.7930, 0.7923]],

        [[0.7940, 0.7938, 0.7950],
         [0.7948, 0.7952, 0.7963],
         [0.7946, 0.7938, 0.7944]]], grad_fn=<SliceBackward0>)

In [137]:
model_ae.test_step((x,y), 0)

/DATASET/envs/o23gauvr/miniconda3/envs/fsc_env/lib/python3.11/site-packages/pytorch_lightning/core/module.py:423: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


tensor(13.7344, grad_fn=<SqrtBackward0>)

## ECS RMSE

In [138]:
y[0,0,25:28,25:28]

tensor([[0.1451, 0.1451, 0.1451],
        [0.1451, 0.1451, 0.1451],
        [0.1451, 0.1451, 0.1451]])

In [139]:
y[0,1,25:28,25:28]

tensor([[0.0195, 0.0197, 0.0198],
        [0.0192, 0.0193, 0.0194],
        [0.0191, 0.0191, 0.0192]])

In [140]:
y[0,1,:,:].max()

tensor(1.)

In [141]:
acoustic_predictor(output)[:,0,45:65,45:65]

tensor([[[0.7321, 0.5722, 0.6005,  ..., 0.7303, 0.6763, 0.6799],
         [0.8483, 0.7252, 0.6828,  ..., 0.7001, 0.7523, 0.7798],
         [0.7172, 0.6908, 0.6954,  ..., 0.6725, 0.7013, 0.7586],
         ...,
         [0.2190, 0.1450, 0.0723,  ..., 0.5276, 0.5486, 0.4346],
         [0.2553, 0.2074, 0.1260,  ..., 0.5049, 0.5433, 0.4814],
         [0.2958, 0.2611, 0.1832,  ..., 0.4940, 0.5288, 0.4822]],

        [[0.7321, 0.5722, 0.6005,  ..., 0.7303, 0.6763, 0.6799],
         [0.8483, 0.7252, 0.6828,  ..., 0.7001, 0.7523, 0.7798],
         [0.7172, 0.6908, 0.6954,  ..., 0.6725, 0.7013, 0.7586],
         ...,
         [0.2190, 0.1450, 0.0723,  ..., 0.5276, 0.5486, 0.4346],
         [0.2553, 0.2074, 0.1260,  ..., 0.5049, 0.5433, 0.4814],
         [0.2958, 0.2611, 0.1832,  ..., 0.4940, 0.5288, 0.4822]],

        [[0.7321, 0.5722, 0.6005,  ..., 0.7303, 0.6763, 0.6799],
         [0.8483, 0.7252, 0.6828,  ..., 0.7001, 0.7523, 0.7798],
         [0.7172, 0.6908, 0.6954,  ..., 0.6725, 0.7013, 0.